## Changes:
- change tower_labels to space-separated list of tags
- final models

## Data - Prep

In [ ]:
    
#initial imports
import os
import random
import numpy as np
import pandas as pd
import json
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import keras
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from matplotlib import pyplot
%matplotlib inline
    
    
# vgg16 imports
import sys
from numpy import load
from keras import backend
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed
from pandas import read_csv
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from numpy import ones
from sklearn.metrics import fbeta_score
from keras import backend

In [ ]:

######################################################################################################
#####################################################################################################
#getting data ready
#create a dataframe containing the names of all of the jpgs in the full dataset
fileList = []
for file in os.listdir('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy'):
    if file.endswith(".jpg"):
        fileList.append(file)

df = pd.DataFrame(fileList)

#also creating an index to use later in extracting the img labels from the json files
index = df.index

#formatting the dataframe
df = df.rename(columns={0: "Id"}) 
df['labels'] = df.Id
df['labels'] = df['labels'].str.replace('.jpg','')
df['tower_wooden'] = ""
df['tower_lattice'] = ""
df['tower_tucohy'] = ""
index = df.index
 ######################################################################################################
#####################################################################################################
for file in os.listdir('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy'):
    if file.endswith(".json"):

        #load file as json object
        filePath = ("C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/"+file)
        f = open(filePath,)
        json_file = json.load(f)
        #get file namme without extension
        file2 = file.replace('.json','')

        #find designated jpeg (row in df)
        #rslt_df = df.loc[df['labels'] == file2]
        apples_indices = index[df['labels'] == file2]

        #parse json file to see if tower_wooden, tower_lattice, or tower_tucohy exists
        #add 0 or 1 to the corresponding dataframe row accordingly

        for element in json_file['shapes']:

            #tower wooden
            bool_wood = any(sd['label']=='tower_wooden' for sd in json_file['shapes'])  
            if bool_wood:
                df.at[apples_indices,'tower_wooden']= 1
            else:
                df.at[apples_indices,'tower_wooden']= 0 

            #tower lattice
            bool_lat = any(sd['label']=='tower_lattice' for sd in json_file['shapes']) 
            if bool_lat:
                df.at[apples_indices,'tower_lattice']= 1
            else:
                df.at[apples_indices,'tower_lattice']= 0

            #tower tucohy
            bool_tuc = any(sd['label']=='tower_tucohy' for sd in json_file['shapes']) 
            if bool_tuc:
                df.at[apples_indices,'tower_tucohy']= 1
            else:
                df.at[apples_indices,'tower_tucohy']= 0
######################################################################################################
#####################################################################################################                  df.at[apples_indices,'tower_tucohy']= 0

#create column containing info for all of the towers
df['towers'] = df[['tower_wooden', 'tower_lattice','tower_tucohy']].values.tolist()
df['tower_labels'] = np.empty((len(df), 0)).tolist()
for i in range(len(df)):

    if df['towers'][i][0] == 1:
        df['tower_labels'][i].append("tower_wooden")

    if df['towers'][i][1] == 1:
        df['tower_labels'][i].append("tower_lattice")

    if df['towers'][i][2] == 1:
        df['tower_labels'][i].append("tower_tucohy")
  ######################################################################################################
#####################################################################################################        
#need to split the data into testing, training, and validating
random.seed(123)
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])    
#reading in all of the training images
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/'+train.iloc[i][0],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
#reading in all of the validation images
valid_image = []
for i in tqdm(range(validate.shape[0])):
    img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/'+validate.iloc[i][0],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    valid_image.append(img)
X_valid = np.array(valid_image)
#reading in all of the test images
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/'+test.iloc[i][0],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X_test = np.array(test_image)
#need to df['DataFrame Column'] = df['DataFrame Column'].astype(float)first convert the column values from strings to floats
df['tower_wooden'] = df['tower_wooden'].astype(int)
df['tower_lattice'] = df['tower_lattice'].astype(int)
df['tower_tucohy'] = df['tower_tucohy'].astype(int)
y = np.array(train.drop(['Id', 'labels','towers','tower_labels'],axis=1))

#validation set y
y_validate = np.array(validate.drop(['Id', 'labels','towers','tower_labels'],axis=1))
#test set y
y_test = np.array(test.drop(['Id', 'labels','towers','tower_labels'],axis=1))


X_train=np.asarray(X).astype(np.int)
y_train=np.asarray(y).astype(np.int)
X_valid=np.asarray(X_valid).astype(np.int)
y_valid=np.asarray(y_validate).astype(np.int)
X_test=np.asarray(X_test).astype(np.int)
y_test=np.asarray(y_test).astype(np.int)


In [ ]:
df['liststring'] = df['tower_labels'].apply(lambda x: ' '.join(map(str, x)))

In [ ]:
df.loc[df['liststring'] == '', 'liststring'] = 'none' 

## mapping for the image labels - 
#### following tutorial: Multi-Label Classification of Satellite Photos of the Amazon Rainforest

In [ ]:
#just want labels and liststring
mapping_df = df[['labels','liststring']]

In [ ]:
#renaming columns to match tutorial
mapping_df = mapping_df.rename(columns={"labels": "image_name", "liststring": "tags"})

In [ ]:
#splitting data into 3 sets
random.seed(123)
train, validate, test = \
              np.split(mapping_df.sample(frac=1, random_state=42), 
                       [int(.6*len(mapping_df)), int(.8*len(mapping_df))])  

In [ ]:
#need to reindex
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
test = test.reset_index(drop=True)


In [ ]:
#getting the image labels 
labels = set()
for i in range(len(mapping_df)):
    tags = mapping_df['tags'][i].split(' ')
    labels.update(tags)

labels = list(labels)

In [ ]:
#getting the mapping
labels_map = {labels[i]:i for i in range(len(labels))}
inv_labels_map = {i:labels[i] for i in range(len(labels))}


In [ ]:
#putting the mapping code into a function

def create_tag_mapping(mapping_csv):
        labels = set()
        for i in range(len(mapping_csv)):
            tags = mapping_csv['tags'][i].split(' ')
            labels.update(tags)
        labels = list(labels)
        labels.sort()
        labels_map = {labels[i]:i for i in range(len(labels))}
        inv_labels_map = {i:labels[i] for i in range(len(labels))}
        return labels_map, inv_labels_map


def create_file_mapping(mapping_csv):
    mapping = dict()
    for i in range(len(mapping_csv)):
        name, tags = mapping_csv['image_name'][i], mapping_csv['tags'][i]
        mapping[name] = tags.split(' ')
    return mapping

# create a one hot encoding for one list of tags
def one_hot_encode(tags, mapping):
    encoding = zeros(len(mapping), dtype='uint8')
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding


# load all images into memory
def load_dataset(file_mapping, tag_mapping, data):
    photos, targets = list(), list()
    for i in tqdm(range(data.shape[0])):
        filename = data.iloc[i][0]
        photo = load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/'+filename+'.jpg',target_size=(400,400,3))
        photo = img_to_array(photo, dtype='uint8')
        tags = file_mapping[filename]
        target = one_hot_encode(tags, tag_mapping)
        photos.append(photo)
        targets.append(target)
    X = asarray(photos, dtype='uint8')
    y = asarray(targets, dtype='uint8')
    return X, y

### loading the train/test/validation sets in new mapping way

In [ ]:
train_mapping_csv = train
train_tag_mapping, _ = create_tag_mapping(train_mapping_csv)
train_file_mapping = create_file_mapping(train_mapping_csv)

trainX, trainY = load_dataset(train_file_mapping, train_tag_mapping, train)
print(trainX.shape, trainY.shape)

In [ ]:
valid_mapping_csv = validate
valid_tag_mapping, _ = create_tag_mapping(valid_mapping_csv)
valid_file_mapping = create_file_mapping(valid_mapping_csv)

validX, validY = load_dataset(valid_file_mapping, valid_tag_mapping, validate)
print(validX.shape, validY.shape)

In [ ]:
test_mapping_csv = test
test_tag_mapping, _ = create_tag_mapping(test_mapping_csv)
test_file_mapping = create_file_mapping(test_mapping_csv)

testX, testY = load_dataset(test_file_mapping, test_tag_mapping, test)
print(testX.shape, testY.shape)

In [ ]:
train_yhat = asarray([ones(trainY.shape[1]) for _ in range(trainY.shape[0])])
valid_yhat = asarray([ones(validY.shape[1]) for _ in range(validY.shape[0])])
test_yhat = asarray([ones(testY.shape[1]) for _ in range(testY.shape[0])])



In [ ]:

#performance metric
def fbeta(y_true, y_pred, beta=2):
    y_pred = backend.clip(y_pred, 0, 1)
    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
    p = tp / (tp + fp + backend.epsilon())
    r = tp / (tp + fn + backend.epsilon())
    bb = beta ** 2
    fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
    return fbeta_score


### VGG- Type Model

In [ ]:

# define vgg-type cnn model
def define_model(in_shape=(400, 400, 3), out_shape=4):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(out_shape, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])
    return model

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = datagen.flow(trainX, trainY, batch_size=32)
valid_it = datagen.flow(validX, validY, batch_size=32)
test_it = datagen.flow(testX, testY, batch_size=32)

model = define_model()


In [ ]:
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=valid_it, validation_steps=len(valid_it), epochs=50, verbose=1)

loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))


In [ ]:
# plot loss
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='validation')

In [ ]:
# plot accuracy (F2)
pyplot.subplot(212)
pyplot.title('Fbeta')
pyplot.plot(history.history['fbeta'], color='blue', label='train')
pyplot.plot(history.history['val_fbeta'], color='orange', label='validation')

## Improve Model 

### dropout regularization

In [ ]:
def define_model(in_shape=(400, 400, 3), out_shape=4):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(out_shape, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])
    return model

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = datagen.flow(trainX, trainY, batch_size=32)
valid_it = datagen.flow(validX, validY, batch_size=32)
test_it = datagen.flow(testX, testY, batch_size=32)

model = define_model()

In [ ]:
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=valid_it, validation_steps=len(valid_it), epochs=50, verbose=1)

loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

In [ ]:
# plot loss
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='validation')


In [ ]:
# plot accuracy (F2)
pyplot.subplot(212)
pyplot.title('Fbeta')
pyplot.plot(history.history['fbeta'], color='blue', label='train')
pyplot.plot(history.history['val_fbeta'], color='orange', label='validation'

#### Image Data Augmentation

In [ ]:
# use the VGG-type define model function
train_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True, vertical_flip=True, rotation_range=90)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True, vertical_flip=True, rotation_range=90)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = train_datagen.flow(trainX, trainY, batch_size=32)
valid_it = valid_datagen.flow(validX, validY, batch_size=32)
test_it = test_datagen.flow(testX, testY, batch_size=32)

model=define_model()

In [ ]:
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=valid_it, validation_steps=len(valid_it), epochs=50, verbose=1)

loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

In [ ]:
# plot loss
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='validation')

In [ ]:
# plot accuracy (F2)
pyplot.subplot(212)
pyplot.title('Fbeta')
pyplot.plot(history.history['fbeta'], color='blue', label='train')
pyplot.plot(history.history['val_fbeta'], color='orange', label='validation'

### Combine dropout and augmentation with more epochs (100)

In [ ]:
## use dropout model
train_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True, vertical_flip=True, rotation_range=90)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True, vertical_flip=True, rotation_range=90)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = train_datagen.flow(trainX, trainY, batch_size=32)
valid_it = valid_datagen.flow(validX, validY, batch_size=32)
test_it = test_datagen.flow(testX, testY, batch_size=32)

model = define_model()

In [ ]:
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=valid_it, validation_steps=len(valid_it), epochs=100, verbose=1)

loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

In [ ]:
# plot loss
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='validation')

In [ ]:
# plot accuracy (F2)
pyplot.subplot(212)
pyplot.title('Fbeta')
pyplot.plot(history.history['fbeta'], color='blue', label='train')
pyplot.plot(history.history['val_fbeta'], color='orange', label='validation')

## Transfer Learning

In [ ]:
def define_model(in_shape=(400, 400, 3), out_shape=4):
    model = VGG16(include_top=False, input_shape=in_shape)
    for layer in model.layers:
        layer.trainable = False
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(out_shape, activation='sigmoid')(class1)

    model = Model(inputs=model.inputs, outputs=output)

    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])
    return model

In [ ]:

datagen = ImageDataGenerator(featurewise_center=True)
datagen.mean = [123.68, 116.779, 103.939]

train_it = datagen.flow(trainX, trainY, batch_size=16)
test_it = datagen.flow(testX, testY, batch_size=16)

model = define_model()

In [ ]:
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=valid_it, validation_steps=len(valid_it), epochs=100, verbose=1)

loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

In [ ]:
# plot loss
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='validation')

In [ ]:
# plot accuracy (F2)
pyplot.subplot(212)
pyplot.title('Fbeta')
pyplot.plot(history.history['fbeta'], color='blue', label='train')
pyplot.plot(history.history['val_fbeta'], color='orange', label='validation')